# Neural Network creation

This script takes results from "aggregated results", and generates neural
networks based on them.


In [ ]:
import sys
from utils.setup import SetupNeuralNetworks

argv  = sys.argv[1:]
# argv  = ["-c", "./nn_config/cfg_NN_Creation.yml"]
# argv  = ["-c", "./nn_config/cfg_NN_Creation_rasp.yml"]
# argv  = ["-c", "./nn_config/cfg_NN_Creation_climate_invariant.yml"]

setup = SetupNeuralNetworks(argv)


One can check both, attributes and methods in **setup** by, for example:\
**dir(setup)\
setup.__dict__** # setup.__dict__.keys() & setup.__dict__.values()

# Create Neural Networks

In [ ]:
from neural_networks.models import generate_models
model_descriptions = generate_models(setup)

**model_descriptions** is a list with all models to be trained. Note, however, that each element in model_description is a class obj. and one can check both, attributes and methods by for example:\
dir(dir(model_descriptions[0]))\
model_descriptions[0].__dict__** # model_descriptions[0].__dict__.keys() & model_descriptions[0].__dict__.values()

# Training

Train all models in the model list and store the results.

In [ ]:
from neural_networks.training import train_all_models

train_all_models(model_descriptions, setup)